# Compute expected counts of mutations from synonymous mutation rates and counts.

Import Python modules:

In [ ]:
import altair as alt

import pandas as pd

Get input variables from `snakemake`:

In [ ]:
if "snakemake" in locals() or "snakemake" in globals():
    clade_founder_nts_csv = snakemake.input.clade_founder_nts_csv
    rates_by_clade_csv = snakemake.input.rates_by_clade
    expected_counts_csv = snakemake.output.expected_counts
else:
    # if running interactively
    clade_founder_nts_csv = "../results/clade_founder_nts/clade_founder_nts.csv"
    rates_by_clade_csv = "../results/synonymous_mut_rates/rates_by_clade.csv"
    expected_counts_csv = "../results/expected_mut_counts/expected_mut_counts.csv"

Read input data:

In [ ]:
rates_by_clade = pd.read_csv(rates_by_clade_csv)

clade_founder_nts = pd.read_csv(clade_founder_nts_csv, low_memory=False)

For each clade, we first compute $T$ that satisfies 
$$N_s = T \sum\limits_{nt_1} s_{nt_1} \sum\limits_{nt_2 \ne nt_1} r_{nt_1\rightarrow nt_2}$$
where $N_s$ is the total number of mutations at 4-fold degenerate synonymous sites observed for the clade, $s_{nt}$ is the number of 4-fold synonymous sites in the clade founder that are nucleotide $nt$, and $r_{nt_1\rightarrow nt_2}$ is the non-normalized rate of mutations from nucleotide $nt_1$ to $nt_2$ at 4-fold degenerate synonymous sites.

The expected number of mutations at each site (under neutrality) from the parental identity of $nt_1$ to some other identity of $nt_2$ is then simply $T \times r_{nt_1\rightarrow nt_2}$, which we will call the normalized rate for that clade:

In [ ]:
# first compute the normalization factor T
normalization_factor = (
    clade_founder_nts.query("four_fold_degenerate")
    .groupby(["clade", "nt"], as_index=False)
    .aggregate(sites_4fold=pd.NamedAgg("site", "nunique"))
    .rename(columns={"nt": "parent_nt"})
    .merge(rates_by_clade, validate="one_to_many")
    .groupby(["clade", "parent_nt", "total_count", "sites_4fold"], as_index=False)
    .aggregate(summand_2=pd.NamedAgg("rate", "sum"))
    .assign(s_nt_1_summand_2=lambda x: x["sites_4fold"] * x["summand_2"])
    .groupby(["clade", "total_count"], as_index=False)
    .aggregate(summand_1=pd.NamedAgg("s_nt_1_summand_2", "sum"))
    .assign(normalization_factor=lambda x: x["total_count"] / x["summand_1"])
    .drop(columns="summand_1")
)

display(normalization_factor)

expected_muts = (
    normalization_factor.merge(clade_founder_nts, on="clade", validate="one_to_many")
    .rename(columns={"nt": "parent_nt"})
    .merge(
        rates_by_clade[["clade", "parent_nt", "mut_type", "rate"]],
        on=["clade", "parent_nt"],
    )
    .assign(expected_count=lambda x: x["normalization_factor"] * x["rate"])
    .rename(columns={"parent_nt": "clade_founder_nt"})[
        [
            "clade",
            "site",
            "clade_founder_nt",
            "gene",
            "codon",
            "codon_position",
            "codon_site",
            "mut_type",
            "four_fold_degenerate",
            "expected_count",
        ]
    ]
)

display(expected_muts)

print(f"Writing expected counts to {expected_counts_csv}")
expected_muts.to_csv(expected_counts_csv, float_format="%.5g", index=False)

Just do a sanity check on the total 4-fold degenerate counts of each mutation type to make sure highly correlated:

In [ ]:
expected_vs_actual_4fold_total = (
    expected_muts.query("four_fold_degenerate")
    .groupby(["clade", "mut_type"], as_index=False)
    .aggregate({"expected_count": "sum"})
    .merge(
        rates_by_clade[["clade", "mut_type", "count"]].rename(
            columns={"count": "actual_count"}
        ),
        validate="one_to_one",
    )
)

display(expected_vs_actual_4fold_total)

(
    alt.Chart(expected_vs_actual_4fold_total)
    .encode(
        x=alt.X("expected_count", scale=alt.Scale(type="log")),
        y=alt.Y("actual_count", scale=alt.Scale(type="log")),
        tooltip=expected_vs_actual_4fold_total.columns.tolist(),
    )
    .mark_point()
    .configure_axis(grid=False)
)